# Install packages

First thing is to install pandas, numpy, scikit-learn, matplotlib, and seaborn. Then below you see where we import the packages in our notebook.

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, ElasticNet, Lasso
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, plot_confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler



# Import the data and start exploring it

In [ ]:

data = pd.read_csv('../Project Retain Alabama/Survey Data and Info/UABformatChange.csv')
alabamaData = pd.read_csv('../Project Retain Alabama/Survey Data and Info/alabamaData.csv')


Some commands you could start with : 
- data.describe()
- data.shape
- data.isnull().sum()
- data['column name'].value_counts()

# Transform the Data for Regression

In [ ]:
data = data.replace(np.NAN, 0)
alabamaData = data.replace(np.NAN, 0)


In [ ]:
# Use get dummies for one hot encoding 
# pd.get_dummies(data.University, prefix='University Code')
readData = pd.get_dummies(data, columns=['Race','StateHS','Major','Major Code', 'Major Code Revised'])
readAlabamaData = pd.get_dummies(data, columns=['Race','StateHS','Major','Major Code', 'Major Code Revised'])
readAlabamaData
# data = pd.concat([data, rd], axis=1)
# data
# print(data)

In [ ]:
# A useful function for changing strings 
# for i in list(data.columns):
#     data[i] = data[i].apply(lambda x: str(x)).apply(lambda x: x.replace('|' , ','))
# data = data[~data.columns.duplicated()]
data.describe()

In [10]:
# renaming columns
pd.options.display.max_columns = 300
pd.options.display.max_rows = 10
# readData
readAlabamaData
# data = data.rename(columns = {'_Virginia': 'Virginia'})
# data
# Regressiona

,University,GradTime,FTPT,Distance,Full,STEM,Alcareerlearn,StayAL,AL Resident,StateHScode,ALHS,Gender,Race Code,Age,Disability,FirstGEn,Rcurric,Rk-12,Rprof,Rfamily,Rfriend,Rmedia,Rextra,Rindustry,Rhighscho,Rcollege,Rprospect,Rjob,Rother,ExploreAL,Eaubrun,Efoley,Ebham,Edemop,Edothan,Eflor,Epayne,Egadsden,Egulf,Ehunts,Emobile,Emontg,Eselma,Etrpy,Etuscal,Eother,ENA,Positive1,Positive2,Positive3,Allive,Alwork,Aloverall,Alfriendly,Aldiverse,Alfaith,Alnature,Alweather,Alsafety,Aljob,Alsalary,Albenefits,Aladvance,Alcostliving,Aleducation,Alpolitical,Alsocial,Alfamily,Aldining,Alevents,Alsporting,Aloutdoor,Ioverall,Ifriendly,Idiverse,Ifaith,Inature,Iweather,Isafety,Ijob,Isalary,Ibenefits,Iadvance,Icostlive,Ieducation,Ipolitical,Isocial,Ifamily,Idining,Ievents,Isporting,Ioutdoor,AL_c_live,A_c_lwork,Al_c_overall,Al_c_friendly,Al_c_diverse,Al_c_faith,Al_c_nature,Al_c_weather,Al_c_safety,Al_c_job,Al_c_salary,Al_c_benefits,Al_c_advance,Al_c_costliving,Al_c_education,Al_c_political,Al_c_social,Al_c_family,Al_c_dining,Al_c_events,Al_c_sporting,Al_c_outdoor,I_c_overall,I_c_friendly,I_c_diverse,I_c_faith,I_c_nature,I_c_weather,I_c_safety,I_c_job,I_c_salary,I_c_benefits,I_c_advance,I_c_costlive,I_c_education,I_c_political,I_c_social,I_c_family,I_c_dining,I_c_events,I_c_sporting,I_c_outdoor,AL Resident.1,StateHS.1,StateHScode.1,ALHS.1,Gender.1,Race.1,Race Code.1,Age.1,Disability.1,FirstGEn.1,Race_American Indian or Alaska Native,Race_Asian,Race_Black or African American,Race_Hispanic or Latino,Race_Native Hawaiian or Other Pacific Islander,Race_Other or Unknown,Race_Two or More Races,Race_White,StateHS_Alabama,StateHS_Alaska,StateHS_Arizona,StateHS_Arkansas,StateHS_California,StateHS_Colorado,StateHS_Connecticut,StateHS_District of Columbia (DC),StateHS_Florida,StateHS_Georgia,StateHS_Illinois,StateHS_Indiana,StateHS_Iowa,StateHS_Kansas,StateHS_Kentucky,StateHS_Louisiana,StateHS_Maryland,StateHS_Massachusetts,StateHS_Michigan,StateHS_Minnesota,StateHS_Mississippi,StateHS_Missouri,StateHS_Montana,StateHS_Nebraska,StateHS_New Mexico,StateHS_New York,StateHS_North Carolina,StateHS_North Dakota,StateHS_Ohio,StateHS_Oklahoma,StateHS_Outside of the U.S.,StateHS_Pennsylvania,StateHS_South Carolina,StateHS_Tennessee,StateHS_Texas,StateHS_Utah,StateHS_Virginia,StateHS_Wisconsin,Major_Agricultural/ Animal Sciences,"Major_Arts, Visual and Performing","Major_Biological Sciences, Biochemistry/ Molecular Biology","Major_Biological Sciences, Biomedical Sciences","Major_Biological Sciences, Cell Biology/ Microbiology/ Immunology","Major_Biological Sciences, General","Major_Biological Sciences, Genetics/ Genomics","Major_Biological Sciences, Marine Biology","Major_Biological Sciences, Neuroscience/ Neurobiology","Major_Biological Sciences, Other",Major_Business Administration/ Management,"Major_Business, Accounting","Major_Business, Economics","Major_Business, Entrepreneurship","Major_Business, Finance/ Banking","Major_Business, Human Resources","Major_Business, International","Major_Business, Logistics/ Supply Chain","Major_Business, Management Information Systems","Major_Business, Management Science/ Analytics","Major_Business, Marketing","Major_Business, Other","Major_Business, Technology Management","Major_Communications, Journalism, Media","Major_Criminal Justice, Emergency Mgmt, Homeland Security","Major_Cultural Studies, African American Studies, Women's Studies","Major_Education, Early Childhood","Major_Education, Elementary","Major_Education, Grades 6-12 English Language Arts","Major_Education, Grades 6-12 Science","Major_Education, Grades 6-12 Social Science","Major_Education, Instructional Technology/ Media","Major_Education, Leadership/ Administration","Major_Education, Music","Major_Education, Other","Major_Education, Physical and Health","Major_Education, Special Education","Major_Engineering, Architectural/ Construction","Major_Engineering, Biomedical/ Medical","Major_Engineering, Civil","Major_Engineering, Computer/ Software","Major_Engineering, 

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# allow multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Model Validation

# 